
# Root Locus Tool

Interactive root‑locus explorer for control systems.

## What it shows
- The tool plots the locus of **`1 + k·P(s) = 0`** for **$k ∈ [0, k_{max}]$**, where **`P(s)`** describes the open loop transfer function.
- Open‑loop **poles** are green crosses; open-loop **zeros** are green circles.
- **K (analysis)** moves the red **closed‑loop poles** along the locus and computes the **the step response specifications** printed on the plot.
- Optional overlays of time domain specifications: **Settling time (2%)** half‑plane and **Overshoot** wedge.

## Prerequisites
- You need to run all code cells first to see the tool.
- As advice we recommend you to hide the code cells when using the tool, such that you need to scroll less. You can do this by clicking the blue bar on the left of a code cell.

## How to use the tool

1. **Pick input type:**
   Choose whether you want to enter the the values manually or want to use a preset.
2. **Presets:** 
   You can pick from some pre-defined presets to see common examples.
   - **TF preset**: Transfer functions related to the problem set.
   - **SS preset**: includes 4th‑order, single‑zero cases for ring‑like locus.
3. **Enter input manually:**
   - **Transfer Function**: Numerator/denominator coefficients, highest power first e.g. `1 3 2` → $s^2+3s+2$.
   - **State Space (SISO)**: Matrices with rows separated by semicolons e.g. `A = 0 1; -4 -0.8` → 
   $\begin{pmatrix}
   0&1\\
   -4&0.8
   \end{pmatrix}   
   $.

4. **Root locus setting:**: Parameters we have used to compute the locus. 
   - **Max. k value to calculate** sets the biggest `k` value which the code is going to plot.
   - **Locus sample points** determines the overall number of points the program is going to consider for the root locus. The bigger the numer of sample points, the more accurate the plot will be, though it takes longer to calculate.
5. **Analyze gain:** Parameters used to check the position of the closed-loop poles, can be turned on and off.
- **k analysis max.** defines the largest value which can be considered for the sweep of `k`. This doesn't influence the plot at all, but helps in dialing the next parameter.
- **k (analysis)** The sweep parameter to see how the closed-loop poles move for changing gain `k`.
6. **Axes limits:** You may manually set the axes limits by ticking the box and selecting your preferred limits.
7. **Specification overlays (optional):**
   - **Settling time (2%)**: shades $\sigma \le -4/T_s$.
   - **Overshoot**: shades the minimum damping ratio $\zeta$ region for your $M_p$, using $M_p=e^{-\zeta\pi/\sqrt{1-\zeta^2}}$.
8. **Specs legend:** All for **$k$ (analysis)**

- **$T_r$ (10–90%) — rise time:** time for the output to go from $10\%$ to $90\%$ of its final value.

- **$T_p$ — peak time:** time to the first (largest) peak.

- **$\%OS$ — percent overshoot:**
  $
  \%OS \;=\; \frac{y_{\text{peak}} - y_{\infty}}{y_{\infty}} \times 100\%.
  $

- **$T_s$ (2\%) — settling time (2% band):** time after which
  $
  \bigl|y(t) - y_{\infty}\bigr| \le 0.02\,\bigl|y_{\infty}\bigr|
  \quad \text{for all } t \ge T_s .
  $

- **$y(\infty)$ — steady-state value** for a unit-step input.

- **$e_{ss}$ — steady-state error** for a unit step:
  $
  e_{ss} \;=\; 1 - y(\infty)\,.
  $

> **Tips**  
> - Coefficients can be **space** or **comma** separated.  
> - Start with small k and increase gradually, to see how the closed loop poles move from the open loop poles to the open loop zeros.  
> - SS inputs are auto‑normalized to SISO shapes.


In [1]:
import numpy as np
import math, warnings
import matplotlib.pyplot as plt
from scipy import signal as sig
from scipy.optimize import linear_sum_assignment

# Widgets (with shim if ipywidgets unavailable)
_HAS_WIDGETS = True
try:
    from ipywidgets import VBox, HBox, Text, FloatText, IntSlider, FloatSlider, Button, Output, Checkbox, Layout, Dropdown, HTML
    from IPython.display import display, Math as _Math, HTML as _HTML
except Exception:
    _HAS_WIDGETS = False
    class _Shim:
        def __init__(self, *a, **k): pass
        def observe(self, *a, **k): pass
        def __setattr__(self, k,v): object.__setattr__(self,k,v)
        def __getattr__(self,k): return None
    VBox = HBox = Text = FloatText = IntSlider = FloatSlider = Button = Output = Checkbox = Layout = Dropdown = HTML = _Shim
    from IPython.display import display, Math as _Math, HTML as _HTML

def parse_coeffs(s):
    toks = [t for t in s.replace(',', ' ').split() if t.strip()!='']
    return np.array([float(t) for t in toks], dtype=float)

def parse_matrix(txt):
    txt = (txt or "").strip()
    if not txt:
        return np.zeros((0,0))
    if '[' in txt:
        import ast
        data = np.array(ast.literal_eval(txt), dtype=float)
    else:
        rows = [r for r in txt.split(';') if r.strip()!='']
        data = np.array([[float(x) for x in r.replace(',', ' ').split()] for r in rows], dtype=float)
    return data

def siso_normalize_shapes(A,B,C,D):
    A = np.atleast_2d(np.array(A, dtype=float))
    n = A.shape[0]
    B = np.array(B, dtype=float).reshape(n, -1)[:, :1]
    C = np.array(C, dtype=float).reshape(-1, n)[:1, :]
    D = np.array(D, dtype=float).reshape(1, 1)
    return A,B,C,D

def ss_to_tf_siso(A,B,C,D):
    num, den = sig.ss2tf(A,B,C,D)
    num = np.array(num).reshape(-1)
    den = np.array(den).reshape(-1)
    def _trim(x):
        i = 0
        while i < len(x)-1 and abs(x[i]) < 1e-12: i += 1
        return x[i:]
    return _trim(num), _trim(den)

def pad_to_same(num, den):
    num = np.array(num, float).ravel()
    den = np.array(den, float).ravel()
    if len(num) < len(den):
        num = np.pad(num, (len(den)-len(num),0))
    elif len(den) < len(num):
        den = np.pad(den, (len(num)-len(den),0))
    return num, den

def _trim_leading_small(p, tol=1e-12):
    p = np.array(p, dtype=float).ravel()
    i = 0
    while i < len(p)-1 and abs(p[i]) < tol:
        i += 1
    return p[i:]

def normalize_poly(num, den, tol=1e-12):
    num = _trim_leading_small(num, tol)
    den = _trim_leading_small(den, tol)
    if len(den)==0 or abs(den[0]) < tol:
        return num, den
    num = num / den[0]; den = den / den[0]
    return num, den

def closed_loop_poly(num, den, K):
    num, den = pad_to_same(num, den)
    return den + K*num

def min_pair_info(roots):
    """Return (min_dist, i,j, root_i, root_j) for closest pair of distinct roots."""
    n = len(roots)
    if n < 2:
        return np.inf, None, None, None, None
    dmin = np.inf
    pair = (None, None)
    for i in range(n):
        for j in range(i+1, n):
            d = abs(roots[i] - roots[j])
            if d < dmin:
                dmin = d
                pair = (i, j)
    i, j = pair
    return dmin, i, j, roots[i], roots[j]

def is_double_root_candidate(num, den, K, tol_dist=6e-1, tol_imag=6e-1, tol_deriv=1.2):
    """
    Check if at gain K there's a candidate double root on the real axis.
    Returns (True/False, info_dict).
    """
    coeffs = closed_loop_poly(num, den, K)
    roots = np.roots(coeffs)
    # find closest pair
    dmin, i, j, r_i, r_j = min_pair_info(roots)
    # check pair is near-real
    imag_ok = (abs(r_i.imag) < tol_imag) and (abs(r_j.imag) < tol_imag)
    # derivative condition at their mean (the double root location)
    # use the average to reduce numerical asymmetry
    s0 = 0.5*(r_i + r_j)
    deriv = np.polyder(coeffs)
    deriv_val = np.polyval(deriv, s0)
    deriv_ok = abs(deriv_val) < tol_deriv
    # combine checks
    is_candidate = (dmin < tol_dist) and imag_ok and deriv_ok
    return is_candidate

def match_roots(prev_roots, roots):
    """
        Improved algorithm which doesn't just greedy nearest-neighbour approach
    """
    cost = np.abs(prev_roots[:, None] - roots[None, :])
    row_ind, col_ind = linear_sum_assignment(cost)
    return roots[col_ind]

def root_locus(num, den, K_vals):
    loci = []
    prev_roots = None
    prev_K = None
    for K in K_vals:
        refinement = 1
        if is_double_root_candidate(num, den, K) == True:
            refinement = 50 #Change refinement value here
        for i in range(0, refinement):
            if prev_K is not None:
                K_i = prev_K + (K-prev_K)*(i+1)/refinement
            else:
                K_i = K
                refinement = 1
            coeffs = closed_loop_poly(num, den, K_i)
            coeffs /= coeffs[0]  # normalize leading term to 1
            roots = np.roots(coeffs)
            # Sort roots by proximity to previous roots to maintain continuity
            """
            if prev_roots is not None and len(roots) == len(prev_roots):
                sorted_roots = np.empty_like(roots)
                available = list(range(len(roots)))
                for i in range(len(prev_roots)):
                    # Find closest available root to prev_roots[i]
                    distances = [np.abs(roots[j] - prev_roots[i]) for j in available]
                    closest_idx = available[np.argmin(distances)]
                    sorted_roots[i] = roots[closest_idx]
                    available.remove(closest_idx)
                roots = sorted_roots
            """
            if prev_roots is not None and len(roots) == len(prev_roots):
                roots = match_roots(prev_roots, roots)
            loci.append(roots)
            prev_roots = roots.copy()
        prev_K = K
    return np.array(loci, dtype=complex)

def poly_to_latex(coefs, var='s'):
    coefs = np.array(coefs, float)
    n = len(coefs)-1
    terms = []
    for i,c in enumerate(coefs):
        p = n-i
        if abs(c) < 1e-12: continue
        sign = '+' if c>=0 else '−'
        mag = abs(c)
        if p>1:
            t = f"{mag:g}{var}^{p}" if abs(mag-1.0)>1e-12 else f"{var}^{p}"
        elif p==1:
            t = f"{mag:g}{var}" if abs(mag-1.0)>1e-12 else f"{var}"
        else:
            t = f"{mag:g}"
        terms.append((sign,t))
    if not terms: return "0"
    first_sign, first_t = terms[0]
    expr = ('' if first_sign=='+' else '−') + first_t
    for s,t in terms[1:]:
        expr += (' + ' if s=='+' else ' − ') + t
    return expr

def mat_to_latex(M):
    M = np.array(M, float)
    rows = [" & ".join(f"{v:g}" for v in row) for row in M]
    return r"\begin{bmatrix} " + r" \\ ".join(rows) + r" \end{bmatrix}"

def _fmt_val(x, unit=''):
    try:
        if x is None or (isinstance(x, float) and (math.isnan(x) or math.isinf(x))):
            return '—'
        return f'{x:.3g}{unit}'
    except Exception:
        return '—'

def _step_specs_from_response(t, y, tol=0.02):
    y = np.asarray(y).ravel(); t = np.asarray(t).ravel()
    y_final = y[-1]
    ess = 1.0 - y_final
    def _first_geq(v, thr):
        idx = np.where(v >= thr)[0]
        return t[idx[0]] if idx.size else np.nan
    t10 = _first_geq(y, 0.1); t90 = _first_geq(y, 0.9)
    Tr = t90 - t10 if np.isfinite(t10) and np.isfinite(t90) else np.nan
    ipk = int(np.argmax(y)); ypk = float(y[ipk]); tpk = float(t[ipk])
    Mp = ((ypk - y_final)/y_final*100.0) if y_final!=0 else np.nan
    if not np.isfinite(Mp) or Mp < 0: Mp = 0.0
    idx_out = np.where(np.abs(y-1.0) > tol)[0]
    if idx_out.size:
        last_out = int(idx_out.max())
        Ts = float(t[last_out+1]) if last_out+1 < len(t) else np.nan
    else:
        Ts = float(t[0])
    return {
        "rise_time_10_90": None if not np.isfinite(Tr) else float(Tr),
        "peak_time": None if not np.isfinite(tpk) else float(tpk),
        "percent_overshoot": None if not np.isfinite(Mp) else float(Mp),
        "settling_time_2pct": None if not np.isfinite(Ts) else float(Ts),
        "steady_state_value": float(y_final),
        "steady_state_error": float(ess),
    }

def compute_step_specs_tf(num, den, t_final=10.0, n=2000):
    num, den = normalize_poly(num, den)
    t = np.linspace(0, t_final, n)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sys = sig.lti(num, den)
        t, y = sig.step(sys, T=t)
    return _step_specs_from_response(t, y), (t, y)


In [2]:

def plot_root_locus(num, den, Kmax=200.0, nk=2000, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(8.6,7.0))
    else:
        fig = ax.figure
    K_vals = np.linspace(0.0, Kmax, int(nk))
    K_vals = np.concatenate([
    np.linspace(0, Kmax//2, int(0.8*nk)),
    np.linspace(Kmax//2, Kmax, int(0.2*nk))
    ])
    loci = root_locus(num, den, K_vals)  # (nk, n)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for i in range(loci.shape[1]):
            ax.plot(loci[:,i].real, loci[:,i].imag, lw=1.5)
    # Poles and zeros at K=0
    p0 = np.roots(den)
    z0 = np.roots(num) if np.any(np.abs(num)>0) else np.array([])
    ax.plot(p0.real, p0.imag, 'x', ms=9, mew=1.8, color='tab:green')
    if z0.size:
        ax.plot(z0.real, z0.imag, 'o', mfc='none', ms=7, mew=1.5, color='tab:green')
    ax.axhline(0, color='k', lw=0.8, alpha=0.4)
    ax.axvline(0, color='k', lw=0.8, alpha=0.4)
    # Axis padding
    xs = loci.real; ys = loci.imag
    xlo, xhi = np.nanpercentile(xs, [20,80]); ylo, yhi = np.nanpercentile(ys, [20,80])
    if not np.isfinite(xlo) or not np.isfinite(xhi) or xlo==xhi: xlo, xhi = -5, 5
    if not np.isfinite(ylo) or not np.isfinite(yhi) or ylo==yhi: ylo, yhi = -5, 5
    padx = 0.38*(xhi-xlo); pady = 0.38*(yhi-ylo)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ax.set_xlim(xlo-padx, xhi+padx)
        ax.set_ylim(ylo-pady, yhi+pady)
    ax.set_xlabel('Re(s)'); ax.set_ylabel('Im(s)'); ax.set_title('Root Locus')
    return K_vals, loci, ax


In [3]:

# --- UI ---
mode = Dropdown(options=['Transfer Function','State Space'], value='Transfer Function', description='Mode')
num_in = Text(description='Numerator', value='1 3')
den_in = Text(description='Denominator', value='1 2 10')

A_in = Text(description='A', value='0 1; -4 -0.8')
B_in = Text(description='B', value='0; 1')
C_in = Text(description='C', value='1 0')
D_in = Text(description='D', value='0')

#Dials for root locus and gain analysis
kmax_in = FloatText(description="Max. k value to calculate", value=1000.0,
                        layout=Layout(width='350px', margin='4px 0 0 8px'),
                        style={'description_width':'150px'})

kmax_in_ana = FloatSlider(description="k analysis max.", min=0.0, max=1000.0, step=1.0, value=200.0,
                        layout=Layout(width='460px', margin='4px 0 0 8px'),
                        style={'description_width':'150px'})

nk_in   = IntSlider(description="Locus sample points", min=1000, max=5000, step=10, value=2000, continuous_update=True,
                        layout=Layout(width='460px', margin='4px 0 0 8px'),
                        style={'description_width':'150px'})

k_sel   = FloatSlider(description="k (analysis)", min=0.0, max=200.0, step=0.1, value=50.0,
                        layout=Layout(width='460px', margin='4px 0 0 8px'),
                        style={'description_width':'150px'})
shw_closedloop = Checkbox(description='Show closed loop poles', value=False)

# Overlays
chk_ts_area = Checkbox(description='Show Settling time (2%) region', value=False)
ts_target   = FloatText(description='Settling time ≤ [s]', value=2.0,
                        layout=Layout(width='360px', margin='4px 0 0 24px'),
                        style={'description_width':'200px'})
chk_mp_area = Checkbox(description='Show Overshoot region', value=False)
mp_target   = FloatText(description='Overshoot ≤ [%]', value=10.0,
                        layout=Layout(width='360px', margin='4px 0 0 24px'),
                        style={'description_width':'200px'})

# Axis limits
chk_manual_lims = Checkbox(description='Manual axis limits', value=False)
xlim_min = FloatText(description='x min', value=-10.0, layout=Layout(width='170px'))
xlim_max = FloatText(description='x max', value=10.0, layout=Layout(width='170px'))
ylim_min = FloatText(description='y min', value=-10.0, layout=Layout(width='170px'))
ylim_max = FloatText(description='y max', value=10.0, layout=Layout(width='170px'))

# TF presets
examples = {
    #"None": {"num":"1 3", "den":"1 2 10", "K":[0,200,600]},
    "— choose —": None,
    "TF: (s+1)/(s^2+2s+2)": {"num":"1 1", "den":"1 2 2", "K":[0,200,600]},
    "TF: (s+5)/((s+1)(s+2)(s+3))": {"num":"1 5", "den":"1 6 11 6", "K":[0,200,600]},
    "TF: (s+1)/(s+12)*s^2*(s^2+4)": {"num":"1 1", "den":"1 12 4 48 0 0", "K":[0,200,6000]},
}
preset = Dropdown(description='TF preset', options=list(examples.keys()), value='— choose —')

# SS presets (some are TF-based; converted to SS on selection)
ss_examples = {
    "— choose —": None,
    # 4 poles, 1 zero (ring-ish loci)
    "SS: 4th order ring-ish (zero at -1)": {"tf_num":[1,1], "tf_den_blocks":[[1,0.4,4],[1,0.4,16]]},
    "SS: 4th order twin pair (zero at -2)": {"tf_num":[1,2], "tf_den_blocks":[[1,0.6,9],[1,0.6,9]]},
    # Smaller ones
    "SS: 2nd order with zero at -1": {"tf_num":[1,1], "tf_den":[1,5,6]},
    "SS: 2nd order with RHP zero (+1)": {"tf_num":[1,-1], "tf_den":[1,2,2]},
    # Originals
    "SS: Double integrator": {"A":"0 1; 0 0", "B":"0; 1", "C":"1 0", "D":"0"},
    "SS: Mass–spring–damper (ζ=0.2, ωn=2)": {"A":"0 1; -4 -0.8", "B":"0; 1", "C":"1 0", "D":"0"},
    "SS: DC motor position (simplified)": {"A":"-2 -5; 1 0", "B":"1; 0", "C":"0 1", "D":"0"},
}
ss_preset = Dropdown(description='SS preset', options=list(ss_examples.keys()), value='— choose —')



ui = VBox([
    HTML("<h3 style='margin:6px 0'>Mode & Presets</h3>"),
    HBox([mode, preset, ss_preset]),
    HTML("<h3 style='margin:6px 0'>Transfer Function</h3>"),
    HBox([num_in, den_in]),
    HTML("<h3 style='margin:6px 0'>State Space</h3>"),
    HBox([A_in, B_in]),
    HBox([C_in, D_in]),
    HTML("<h3 style='margin:6px 0'>Root Locus Settings</h3>"),
    HBox([kmax_in]),
    HBox([nk_in]),
    HTML("<h3 style='margin:6px 0'>Analyze Gain</h3>"),
    HBox([shw_closedloop]),
    HBox([kmax_in_ana]),
    HBox([k_sel]),
    HTML("<h3 style='margin:6px 0'>Spec. Overlays</h3>"),
    HBox([chk_ts_area, ts_target]),
    HBox([chk_mp_area, mp_target]),
    HTML("<h3 style='margin:6px 0'>Axis Limits</h3>"),
    HBox([chk_manual_lims]),
    HBox([xlim_min, xlim_max]),
    HBox([ylim_min, ylim_max]),
])

def _mat_to_str(M):
    import numpy as _np
    M = _np.array(M, float)
    return "; ".join(" ".join(f"{v:g}" for v in row) for row in M)
# Update kmax_in slider max when kmax_max_in changes
def _sync_kmax_max(change):
    try:
        new_max = float(kmax_in.value)
        if new_max > 0:
            kmax_in_ana.max = new_max
            if float(kmax_in_ana.value) > new_max:
                kmax_in_ana.value = new_max
    except Exception:
        pass
kmax_in.observe(_sync_kmax_max, names='value')
_sync_kmax_max(None)

# Keep k_sel within [0, kmax]
def _sync_k_sel(change):
    try:
        k_sel.max = float(kmax_in_ana.value)
        if float(k_sel.value) > float(kmax_in_ana.value):
            k_sel.value = float(kmax_in_ana.value)
    except Exception:
        pass
kmax_in_ana.observe(_sync_k_sel, names='value')
_sync_k_sel(None)

tf_ss_out = Output()
plot_out = Output()
def update_plot(*_):
    with tf_ss_out:
        tf_ss_out.clear_output(wait=True)
        # Parse plant
        if mode.value == 'Transfer Function':
            num = parse_coeffs(num_in.value)
            den = parse_coeffs(den_in.value)
            # Show TF (once)
            try:
                
                N = poly_to_latex(num, 's'); D = poly_to_latex(den, 's')
                display(HTML("<div style='height:16px;'></div>"))  # top spacing
                display(_Math(r"\displaystyle P(s) = \frac{"+N+r"}{"+D+r"}"))
                display(HTML("<div style='height:10px;'></div>"))  # top spacing
            except Exception:
                pass
        else:
            A = parse_matrix(A_in.value); B = parse_matrix(B_in.value); C = parse_matrix(C_in.value); Dm = parse_matrix(D_in.value if D_in.value.strip() else '0')
            A,B,C,Dm = siso_normalize_shapes(A,B,C,Dm)
            # Show SS
            try:
                ss_ltx = (
                    r"\begin{aligned} "
                    + "A &= " + mat_to_latex(A) + r" \\ "
                    + "B &= " + mat_to_latex(B) + r" \\ "
                    + "C &= " + mat_to_latex(C) + r" \\ "
                    + "D &= " + mat_to_latex(Dm) + r" \end{aligned}"
                )
                display(HTML("<div style='height:16px;'></div>"))  # top spacing
                display(_Math(ss_ltx))
                display(HTML("<div style='height:10px;'></div>"))  # top spacing
            except Exception:
                pass
            num, den = ss_to_tf_siso(A,B,C,Dm)
    with plot_out:
        plot_out.clear_output(wait=True)
        # Plot locus
        K_vals, loci, ax = plot_root_locus(num, den, Kmax=kmax_in.value, nk=nk_in.value)
        # Highlight closed-loop poles at selected K
        try:
            Ksel = float(k_sel.value)
            num_cl, den_cl = pad_to_same(num, den)
            cl_poly = den_cl + Ksel * num_cl
            cl_poles = np.roots(cl_poly)
        except Exception:
            pass

        try:
            if shw_closedloop.value:
                ax.plot(cl_poles.real, cl_poles.imag, 'x', ms=10, mew=1.8, color='tab:red')
        except Exception:
            pass

        # Overlays
        try:
            import matplotlib.patches as patches
            xlim = ax.get_xlim(); ylim = ax.get_ylim()
            xr = (min(xlim[0], xlim[1]), max(xlim[0], xlim[1]))
            yr = (min(ylim[0], ylim[1]), max(ylim[0], ylim[1]))
            R = max(abs(xr[0]), abs(xr[1]), abs(yr[0]), abs(yr[1]))
            if chk_ts_area.value:
                Tsm = float(ts_target.value)
                if Tsm > 0:
                    sigma = -4.0 / Tsm
                    ax.axvspan(min(xr[0], sigma), sigma, alpha=0.10, facecolor='tab:green', label=f'Settling time ≤ {Tsm:g}s (2%)')
                    ax.axvline(sigma, linestyle='--', linewidth=1.0, alpha=0.5, color='tab:green')
            if chk_mp_area.value:
                Mp_pct = max(0.0, float(mp_target.value)); Mp = Mp_pct/100.0
                if 0.0 < Mp < 1.0:
                    lnM = np.log(Mp)
                    zeta = np.sqrt( lnM**2 / (np.pi**2 + lnM**2) )
                    half = np.arccos(zeta)
                    theta1 = 180 - half*180/np.pi; theta2 = 180 + half*180/np.pi
                    wedge = patches.Wedge(center=(0,0), r=R*1.2, theta1=theta1, theta2=theta2, facecolor='tab:orange', alpha=0.10, label=f'Overshoot ≤ {Mp_pct:g}% (ζ≥{zeta:.2f})')
                    ax.add_patch(wedge)
                    ax.plot([0, -R*np.cos(half)], [0,  R*np.sin(half)], linestyle='--', alpha=0.5, linewidth=1.0, color='tab:orange')
                    ax.plot([0, -R*np.cos(half)], [0, -R*np.sin(half)], linestyle='--', alpha=0.5, linewidth=1.0, color='tab:orange')
        except Exception:
            pass

        # Specs annotation at Ksel
        try:
            specs, _ = compute_step_specs_tf(Ksel*num_cl, den_cl + Ksel*num_cl, t_final=10.0, n=2000)
            info = (
                f"k = {_fmt_val(Ksel)}\n"
                f"Tr(10–90%): {_fmt_val(specs.get('rise_time_10_90'),' s')}\n"
                f"Tp: {_fmt_val(specs.get('peak_time'),' s')}\n"
                f"%OS: {_fmt_val(specs.get('percent_overshoot'),' %')}\n"
                f"Ts(2%): {_fmt_val(specs.get('settling_time_2pct'),' s')}\n"
                f"y(∞): {_fmt_val(specs.get('steady_state_value'))}\n"
                f"e_ss: {_fmt_val(specs.get('steady_state_error'))}"
            )
            ax.text(1.06, 0.75, info, transform=ax.transAxes, ha='left', va='bottom',
                    fontsize=10, family='monospace',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.85, edgecolor='0.6'))
        except Exception:
            pass

        # Apply manual axis limits if enabled
        try:
            if chk_manual_lims.value:
                ax.set_xlim(float(xlim_min.value), float(xlim_max.value))
                ax.set_ylim(float(ylim_min.value), float(ylim_max.value))
        except Exception:
            pass

        display(ax.figure)
        plt.close(ax.figure)


# Watchers
for w in [mode, num_in, den_in, A_in, B_in, C_in, D_in, kmax_in, shw_closedloop, nk_in, k_sel, chk_ts_area, ts_target, chk_mp_area, mp_target, chk_manual_lims, xlim_min, xlim_max, ylim_min, ylim_max]:
    try:
        w.observe(lambda *_: update_plot(), names='value')
    except Exception:
        pass

def _apply_preset(change):
    ex = examples.get(change['new'])
    if ex:
        num_in.value = ex['num']; den_in.value = ex['den']
        kmax_in.value = ex['K'][1]; nk_in.value = ex['K'][2]
preset.observe(_apply_preset, names='value')

def _apply_ss_preset(change):
    ex = ss_examples.get(change['new'])
    if not ex: return
    if 'tf_num' in ex:
        try:
            # Build denominator from blocks or full coef list
            if 'tf_den_blocks' in ex and ex['tf_den_blocks']:
                den = np.array([1.0])
                for blk in ex['tf_den_blocks']:
                    den = np.convolve(den, np.array(blk, dtype=float))
            else:
                den = np.array(ex['tf_den'], dtype=float)
            num = np.array(ex['tf_num'], dtype=float)
            # pad numerator to denominator length
            if num.size < den.size:
                num = np.pad(num, (den.size - num.size, 0))
            A,B,C,Dm = sig.tf2ss(num, den)
            A_in.value = _mat_to_str(A); B_in.value = _mat_to_str(B); C_in.value = _mat_to_str(C); D_in.value = _mat_to_str(Dm)
        except Exception:
            pass
    else:
        A_in.value = ex['A']; B_in.value = ex['B']; C_in.value = ex['C']; D_in.value = ex['D']
ss_preset.observe(_apply_ss_preset, names='value')
display(VBox([ui, tf_ss_out, plot_out]))
update_plot()
